# Projeto de Aprendizagem Automática II

## Procura de Exoplanetas no Espaço através da Emissão de Luz de Estrelas

### Importação de Bibliotecas

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model
import keras.backend as K
from keras.layers import *
from keras.optimizers import Adam
import matplotlib            
import matplotlib.pyplot as plt

### Carregamento dos Dados

In [3]:
treino = pd.read_csv("../../../Dados/dados_treino.csv")
teste = pd.read_csv("../../../Dados/dados_teste.csv")

### Preparação dos Dados

O primeiro passo é a preparação dos dados para o tipo de rede que se está a criar. Assim, tendo em conta que será utilizado o método *GridSearchCV*, este apenas recebe um *array* de *features* e outro de *labels*, ambos unidimensionais, não pode ser efetuado o *reshape* dos dados neste passo.

In [4]:
Y = treino['LABEL']
X = treino.loc[:, treino.columns != 'LABEL']
X_train = X.values#.reshape(-1, X.shape[1], 1)

Yt = teste['LABEL']
Xt = teste.loc[:, teste.columns != 'LABEL']
X_test = Xt.values#.reshape(-1, Xt.shape[1], 1)

# One Hot Encoding
y_train = to_categorical(Y.values)
y_train = y_train[:, 1:]

y_test = to_categorical(Yt.values)
y_test = y_test[:, 1:]

### Modelo

Mais uma vez, tendo em conta a utilização do método *GridSearchCV*, foi necessária a definição de uma função para criar e retornar o modelo, com base em certos parâmetros, nomeadamente a taxa de aprendizagem e a probabilidade de *dropout*. Além disso, o modelo difere do original na medida em que possui uma camada inicial de *reshape* dos dados, devido às restrições no formato de dados do método de pesquisa em grelha.

In [5]:
from keras.models import Sequential
def create_model(learn_rate = 0.001, dropout = 0.25):
    model = Sequential()
    model.add(Reshape((3198, 1)))
    model.add(Conv1D(filters=16, kernel_size=16, activation='relu', input_shape=(3198, 1)))
    model.add(MaxPool1D())
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(2, activation="softmax"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learn_rate), metrics=['accuracy'])
    
    return model

Para criar um modelo, é necessário instanciar uma variável como um *KerasClassifier*, sendo construído com base na função definida anteriormente para gerar um modelo.

In [6]:
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=1)

De forma a otimizar os parâmetros do modelo, estes devem ser definidos na forma de dicionário. Neste caso, optou-se pelo teste de várias combinações, vendo qual a que melhores resultados traria ao problema. Assim, foram testados os impactos de parâmetros como *batch_size*, número de épocas, taxa de aprendizagem e probabilidade de *dropout*.

In [17]:
parametros = {
    'batch_size': [64, 128, 256],
    'epochs': [10, 15, 20],
    'learn_rate': [0.001, 0.01, 0.0005, 0.0001, 0.00005, 0.00001],
    'dropout' : [0.0, 0.1, 0.2, 0.25]
}

De modo a testar cada caso na grelha, foi definida uma métrica personalizada, permitindo comparar os valores reais com os calculados, retornando a *accuracy* dos cálculos.

In [8]:
from sklearn.metrics import accuracy_score
def custom_metric(y_true, y_predicted):
    return accuracy_score(y_true.argmax(axis=-1), y_predicted)

Além da definição da função para a métrica, é, ainda, fundamental definir como *scorer* a função e qual a orientação ideal para os resultados, ou seja, se quanto mais elevado melhor, ou o oposto. Neste caso, quanto mais elevada a *accuracy*, melhor o modelo testado.

In [9]:
from sklearn.metrics import make_scorer
custom_score = make_scorer(custom_metric, greater_is_better=True)

Como mencionado anteriormente, foi tirado proveito do método *GridSearchCV*, permitindo uma pesquisa em grelha dentro das combinações de parâmetros pretendidas, em que exista uma validação em cruz. Neste caso, optou-se por uma validação cruzada de 3 subconjuntos, devido ao elevado peso computacional dos modelos, sendo utilizada como métrica de *scoring* a função definida em cima.

In [18]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=parametros, n_jobs=-1, cv=3, scoring=custom_score, verbose=2)

Tendo a grelha definida e pronta a efetuar a procura, apenas é necessário aplicar o método *fit* com o conjunto de dados de treino. Não faria sentido aplicar a validação cruzada aos dados de teste juntamente com os de treino, já que o modelo ficaria demasiado ajustado aos dados, perdendo a capacidade de classificar corretamente novos casos.

In [19]:
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 48.0min
C:\Users\jonyp\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 307.9min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 669.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 1104.6min
[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed: 1131.0min finished


Epoch 1/20
6565/6565 [==============================] - 11s 2ms/step - loss: 0.1449 - accuracy: 0.9438
Epoch 2/20
6565/6565 [==============================] - 10s 2ms/step - loss: 0.0123 - accuracy: 0.9959
Epoch 3/20
6565/6565 [==============================] - 10s 2ms/step - loss: 0.0099 - accuracy: 0.9973
Epoch 4/20
6565/6565 [==============================] - 10s 2ms/step - loss: 0.0053 - accuracy: 0.9986
Epoch 5/20
6565/6565 [==============================] - 10s 2ms/step - loss: 0.0048 - accuracy: 0.9983
Epoch 6/20
6565/6565 [==============================] - 10s 2ms/step - loss: 0.0054 - accuracy: 0.9986
Epoch 7/20
6565/6565 [==============================] - 11s 2ms/step - loss: 0.0046 - accuracy: 0.9985
Epoch 8/20
6565/6565 [==============================] - 10s 2ms/step - loss: 0.0044 - accuracy: 0.9986
Epoch 9/20
6565/6565 [==============================] - 11s 2ms/step - loss: 0.0060 - accuracy: 0.9973
Epoch 10/20
6565/6565 [==============================] - 10s 2ms/step - l

Utilizando o método *get_params* é possível observar os vários parâmetros associados à pesquisa efetuada.

In [20]:
grid_result.get_params()

{'cv': 3,
 'error_score': nan,
 'estimator__verbose': 1,
 'estimator__build_fn': <function __main__.create_model(learn_rate=0.001, dropout=0.25)>,
 'estimator': <keras.wrappers.scikit_learn.KerasClassifier at 0x1a4ce2ddd88>,
 'iid': 'deprecated',
 'n_jobs': -1,
 'param_grid': {'batch_size': [64, 128, 256],
  'epochs': [10, 15, 20],
  'learn_rate': [0.001, 0.01, 0.0005, 0.0001, 5e-05, 1e-05],
  'dropout': [0.0, 0.1, 0.2, 0.25]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': make_scorer(custom_metric),
 'verbose': 2}

Como se pode ver de seguida, os melhores parâmetros de entre os testados são *batch_size* com tamanho 256, ou seja, a cada iteração, a rede é treinada com 256 registos. Além disso, o número de épocas que permitiu os melhores resultados foi 20. No que toca aos parâmetros da função de criação do modelo, o valor de *dropout* foi de 20%, querendo isto dizer que a cada fase existe uma probabilidade de 20% de cada neurónio ser desativado. Já a taxa de aprendizagem tomou o valor de 0.0001.

In [21]:
grid_result.best_params_

{'batch_size': 256, 'dropout': 0.2, 'epochs': 20, 'learn_rate': 0.0001}

O melhor resultado obtido para a pesquisa em grelha, com os parâmetros acima mencionados, permitiu uma *accuracy* de sensivelmente 96.1%. Note-se que este valor está associado a uma validação cruzada com 3 subconjuntos de dados.

In [22]:
grid_result.best_score_

0.9606951855282172

### Teste

Conhecidos os melhores parâmetros para este modelo, é necessário ver de que forma se comporta na classificação do conjunto de teste.

In [27]:
preds = grid_result.predict(X_test)

570/570 [==============================] - 0s 623us/step


De modo a comparar os resultados, de forma mais fácil, com as *labels* reais, as previsões foram convertidas em registos categóricos.

In [29]:
preds = to_categorical(preds)

Observando as métricas obtidas pela predição dos registos de teste, é notório que a precisão da classe minoritária desceu quando em comparação com o modelo original (o qual tinha valor de 67%). Além disso, também o *recall* baixou, passando de 80% para 60%.

In [30]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_test, preds))
print("accuracy:", accuracy_score(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       565
           1       0.60      0.60      0.60         5

   micro avg       0.99      0.99      0.99       570
   macro avg       0.80      0.80      0.80       570
weighted avg       0.99      0.99      0.99       570
 samples avg       0.99      0.99      0.99       570

accuracy: 0.9929824561403509


Observando as matrizes de confusão e, mais uma vez, comparando com o modelo original, verifica-se que foram classificados 3 dos 5 sistemas da classe minoritária corretamente, ao invés de 4 em 5 registos, como originalmente alcançado. No que toca a classificações erradas de sistemas da classe maioritária, em ambos os modelos foram classificados incorretamente apenas 2 registos.

In [31]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(y_test, preds)

array([[[  3,   2],
        [  2, 563]],

       [[563,   2],
        [  2,   3]]], dtype=int64)

Em jeito de conclusão, a otimização do modelo não trouxe uma melhoria na classificação dos registos da classe minoritária, como esperado. No entanto, tendo em conta a quantidade reduzida de registos da classe minoritária, a diferença entre este modelo e o original é bastante pequena. Desta forma, não é possível afirmar com certeza que um modelo é superior ao outro. Apesar disso, tendo em conta a utilização de uma pesquisa em grelha com validação cruzada sobre o conjunto de treino, os parâmetros deste modelo permitiram a obtenção de melhores resultados, sendo possível afirmar que este modelo permite uma melhor classificação no panorama geral.